In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression as LR, Ridge
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.metrics import r2_score as r2, mean_squared_error as MSE
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv('train.csv')

In [3]:
data=df.copy()

In [4]:
#data.head()

Гипотеза 1
Удалаяем признаки, которые имеют пропуски

In [5]:
data.drop(['Healthcare_1'], axis=1, inplace=True)

In [6]:
#data.info()

Гипотеза 2
Изменеям выбросы в годах постройки домов

In [7]:
data.loc[data.HouseYear>2020]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1497,10814,109,1.0,37.265070,20.239714,9.0,9,12.0,20052011,0.136330,B,B,30,6141,10,3,6,B,254084.534396
4189,11607,147,2.0,44.791836,28.360393,5.0,4,9.0,4968,0.319809,B,B,25,4756,16,5,8,B,243028.603096


In [8]:
data.loc[1497,'HouseYear']=2008
data.loc[4189,'HouseYear']=1968

Гипотеза 3
Заменяем колличество квартир больше 6, на колличесво, которое в соответвует площади квартиры

In [9]:
data.loc[data.Rooms>6]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002,0.090799,B,B,74,19083,2,5,15,B,317265.323792
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014,0.007122,B,B,1,264,0,0,1,B,78364.616704
8849,14865,9,10.0,60.871266,38.420681,10.0,3,2.0,1994,0.161532,B,B,25,5648,1,2,4,B,172329.270863


In [10]:
data.loc[377,'Rooms']=2
data.loc[1454,'Rooms']=1
data.loc[8849,'Rooms']=2

Гипотеза 4
Считаем, что в картире не может быть 0 комнат и заменям такие квартиры на однокомнатные

In [11]:
data.loc[data['Rooms']==0,'Rooms']=1

In [12]:
#data.describe()

Гипотеза 4 заполняем пропущенные значения 'LifeSquare' медианными значениями

In [13]:
def fillna_lifesquare(df,df_source):
    #df['LifeSquare']=df['LifeSquare'].fillna(df_source['LifeSquare'].mean())
    df['LifeSquare']=df['LifeSquare'].fillna(df_source['LifeSquare'].median())
    return df

In [14]:
data=fillna_lifesquare(data,data)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
Id               10000 non-null int64
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
dtypes: float64(7), int64(9), object(3)
memory usage: 1.4+ MB


In [16]:
data.LifeSquare.value_counts()

32.781260    2114
35.812832       1
70.238395       1
35.213655       1
23.656629       1
4.289714        1
41.141156       1
29.406640       1
44.889920       1
58.845345       1
19.457465       1
4.502443        1
21.304556       1
23.535141       1
35.086787       1
47.204497       1
58.218079       1
19.224066       1
21.613298       1
25.688943       1
31.814202       1
65.858689       1
21.661180       1
78.817292       1
41.515316       1
47.820840       1
21.701365       1
19.363106       1
33.794672       1
48.373755       1
             ... 
47.748439       1
68.503454       1
29.027271       1
28.304102       1
27.383789       1
31.700385       1
45.233108       1
26.715238       1
31.818619       1
28.620671       1
41.150288       1
43.501569       1
40.955051       1
32.542126       1
31.037172       1
27.640733       1
87.018830       1
32.356012       1
45.470444       1
28.411852       1
32.078970       1
33.230940       1
38.146985       1
46.375497       1
59.170833 

In [17]:
data.drop(['Id'],axis=1, inplace=True)

Гипотеза №5 вводим признак-средняя цена квартиры в зависимости от колличесва комнат и района

In [18]:

stats1=data.groupby(['DistrictId','Rooms'],as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price'})
stats1.head(25)

,DistrictId,Rooms,mean_price
0,0,1.0,141262.601991
1,0,2.0,197998.146681
2,0,3.0,304170.518823
3,1,1.0,147186.814704
4,1,2.0,198777.278269
5,1,3.0,246262.924161
6,1,4.0,277290.705058
7,2,1.0,177871.758149
8,2,2.0,243022.112038
9,2,3.0,342699.342039


In [19]:
def join_stats1(df,stats1,source_df,mode='train'):
    df=pd.merge(df,stats1,on=['DistrictId','Rooms'],how='left')
    if mode=='test':
        df['mean_price']=df['mean_price'].fillna(source_df['mean_price'].mean())
    return df


In [20]:
data=join_stats1(data,stats1,data)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 19 columns):
DistrictId       10000 non-null int64
Rooms            10000 non-null float64
Square           10000 non-null float64
LifeSquare       10000 non-null float64
KitchenSquare    10000 non-null float64
Floor            10000 non-null int64
HouseFloor       10000 non-null float64
HouseYear        10000 non-null int64
Ecology_1        10000 non-null float64
Ecology_2        10000 non-null object
Ecology_3        10000 non-null object
Social_1         10000 non-null int64
Social_2         10000 non-null int64
Social_3         10000 non-null int64
Helthcare_2      10000 non-null int64
Shops_1          10000 non-null int64
Shops_2          10000 non-null object
Price            10000 non-null float64
mean_price       10000 non-null float64
dtypes: float64(8), int64(8), object(3)
memory usage: 1.5+ MB


In [22]:
data.head()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price,mean_price
0,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,0,11,B,184966.930730,213160.076297
1,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,1,16,B,300009.450063,303643.563728
2,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,1,3,B,220925.908524,259938.729121
3,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,0,5,B,175616.227217,172368.882269
4,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2,4,B,150226.531644,155748.415754


Гипотеза №6 Используем только числовые признаки

In [23]:
features=data[['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor',
       'HouseYear', 'Ecology_1', 'Social_1','Social_2', 'Social_3', 'Helthcare_2', 'Shops_1','mean_price']]
target=data['Price']

In [24]:
features.head()

,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,mean_price
0,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,33,7976,5,0,11,213160.076297
1,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,46,10309,1,1,16,303643.563728
2,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,34,7759,0,1,3,259938.729121
3,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,23,5735,3,0,5,172368.882269
4,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,35,5776,1,2,4,155748.415754


In [25]:
target.head()

0    184966.930730
1    300009.450063
2    220925.908524
3    175616.227217
4    150226.531644
Name: Price, dtype: float64

In [26]:
X_train,X_valid,y_train,y_valid=train_test_split(features,target,test_size=0.3, random_state=54)

In [27]:
model1=LR()
model2=Ridge()


In [28]:
params={
        'min_samples_leaf':[3,5,7],
        'n_estimators':[700,800,900],
        'max_depth':[6, 7, 8],
        'n_jobs':[-1]
        }


In [29]:
regr=GridSearchCV(RF(),param_grid=params,cv=2)

In [30]:
%%time
model1.fit(X_train,y_train)

Wall time: 11 ms


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [31]:
%%time
model2.fit(X_train,y_train)

Wall time: 3.99 ms


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [32]:
%%time
regr.fit(X_train,y_train)

Wall time: 1min 3s


GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'min_samples_leaf': [3, 5, 7], 'n_estimators': [700, 800, 900], 'max_depth': [6, 7, 8], 'n_jobs': [-1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
regr.best_params_

{'max_depth': 8, 'min_samples_leaf': 3, 'n_estimators': 800, 'n_jobs': -1}

In [34]:
regr.best_score_

0.7486937618788023

Используем следующие параметры:
n_estimators=800,max_depth=8,min_samples_leaf=5

In [35]:
model3=RF(n_estimators=800,max_depth=8,min_samples_leaf=5,n_jobs= -1)

In [36]:
%%time
model3.fit(X_train,y_train)

Wall time: 1.51 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [37]:
y_pred_1=model1.predict(X_valid)
y_pred_2=model2.predict(X_valid)
y_pred_3=model3.predict(X_valid)

In [38]:
print('Линейная регрессия',MSE(y_valid,y_pred_1),r2(y_valid,y_pred_1))

Линейная регрессия 3168432910.6264772 0.6291347900630713


In [39]:
print('Проверка переобучения',r2(y_train,model1.predict(X_train)))

Проверка переобучения 0.6970659617659198


In [40]:
print('Ridge регрессия',MSE(y_valid,y_pred_2),r2(y_valid,y_pred_2))

Ridge регрессия 3168595203.350457 0.6291157936926737


In [41]:
print('Проверка переобучения',r2(y_train,model2.predict(X_train)))

Проверка переобучения 0.6970659416388293


In [42]:
print('Случайный лес',MSE(y_valid,y_pred_3),r2(y_valid,y_pred_3))

Случайный лес 2206938744.955233 0.7416777239559945


In [43]:
print('Проверка переобучения',r2(y_train,model3.predict(X_train)))

Проверка переобучения 0.8293227191687502


In [44]:
r2(y_train,model3.predict(X_train))-r2(y_valid,y_pred_3)

0.0876449952127557

# Загружаем тестовые данные

In [56]:
test=pd.read_csv('test.csv')

In [57]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       3959 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [58]:
test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,8.632000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.483228,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,1.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,12.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,78.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [59]:
test.loc[test['Rooms']==0,'Rooms']=1

In [60]:
test=fillna_lifesquare(test,data)

In [61]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
dtypes: float64(7), int64(9), object(3)
memory usage: 742.3+ KB


In [62]:
test=join_stats1(test,stats1,data,mode='test')

In [64]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 20 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
mean_price       5000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 820.3+ KB


In [65]:
temp=test[['Rooms', 'Square', 'LifeSquare', 'KitchenSquare', 'Floor', 'HouseFloor',
       'HouseYear', 'Ecology_1', 'Social_1','Social_2', 'Social_3', 'Helthcare_2', 'Shops_1','mean_price']]

In [66]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 20 columns):
Id               5000 non-null int64
DistrictId       5000 non-null int64
Rooms            5000 non-null float64
Square           5000 non-null float64
LifeSquare       5000 non-null float64
KitchenSquare    5000 non-null float64
Floor            5000 non-null int64
HouseFloor       5000 non-null float64
HouseYear        5000 non-null int64
Ecology_1        5000 non-null float64
Ecology_2        5000 non-null object
Ecology_3        5000 non-null object
Social_1         5000 non-null int64
Social_2         5000 non-null int64
Social_3         5000 non-null int64
Healthcare_1     2623 non-null float64
Helthcare_2      5000 non-null int64
Shops_1          5000 non-null int64
Shops_2          5000 non-null object
mean_price       5000 non-null float64
dtypes: float64(8), int64(9), object(3)
memory usage: 820.3+ KB


In [67]:
test['Price']=model3.predict(temp)

In [68]:
test[['Id','Price']].to_csv('VMikhailovskiy_predictions.csv',index=False)